In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# !pip install ucimlrepo shap --quiet
# !pip install --upgrade cubist --quiet
# !pip install --upgrade optuna --quiet

In [3]:
from ucimlrepo import fetch_ucirepo
import numpy as np
import pandas as pd
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
import shap
from cubist import Cubist

# fetch dataset
steel_industry_energy_consumption = fetch_ucirepo(id=851)

In [4]:
df = pd.DataFrame(steel_industry_energy_consumption.data.original)

In [5]:
df.sample(n=10)

,date,Usage_kWh,Lagging_Current_Reactive.Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2(tCO2),Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM,WeekStatus,Day_of_week,Load_Type
23703,04/09/2018 22:00,3.49,0.00,16.88,0.00,100.00,20.25,79200,Weekday,Tuesday,Medium_Load
19387,21/07/2018 23:00,2.81,0.00,5.22,0.00,100.00,47.40,82800,Weekend,Saturday,Medium_Load
2912,31/01/2018 08:15,42.62,30.28,0.00,0.02,81.52,100.00,29700,Weekday,Wednesday,Light_Load
9080,05/04/2018 14:15,49.10,9.43,0.00,0.02,98.21,100.00,51300,Weekday,Thursday,Maximum_Load
30071,10/11/2018 06:00,5.51,7.27,0.00,0.00,60.40,100.00,21600,Weekend,Saturday,Light_Load
5059,22/02/2018 17:00,79.81,35.78,0.00,0.04,91.25,100.00,61200,Weekday,Thursday,Medium_Load
29537,04/11/2018 16:30,71.53,18.07,0.00,0.03,96.95,100.00,59400,Weekend,Sunday,Medium_Load
11105,26/04/2018 16:30,63.54,29.81,0.00,0.03,90.53,100.00,59400,Weekday,Thursday,Maximum_Load
25082,19/09/2018 06:45,2.66,5.00,0.00,0.00,46.97,100.00,24300,Weekday,Wednesday,Light_Load
4469,16/02/2018 13:30,3.49,0.00,16.81,0.00,100.00,20.33,48600,Weekday,Friday,Light_Load


In [6]:
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y %H:%M')

In [7]:
pd.DataFrame(steel_industry_energy_consumption.variables)

,name,role,type,demographic,description,units,missing_values
0,date,Other,Date,None,None,None,no
1,Usage_kWh,Feature,Continuous,None,Industry Energy Consumption,kWh,no
2,Lagging_Current_Reactive.Power_kVarh,Feature,Continuous,None,None,kVarh,no
3,Leading_Current_Reactive_Power_kVarh,Feature,Continuous,None,None,kVarh,no
4,CO2(tCO2),Feature,Continuous,None,None,ppm,no
5,Lagging_Current_Power_Factor,Feature,Continuous,None,None,%,no
6,Leading_Current_Power_Factor,Feature,Continuous,None,None,%,no
7,NSM,Feature,Integer,None,None,s,no
8,WeekStatus,Feature,Categorical,None,Weekend (0) or a Weekday(1),None,no
9,Day_of_week,Feature,Categorical,None,"Sunday, Monday, ..., Saturday",None,no


In [8]:
df.dtypes

date                                    datetime64[ns]
Usage_kWh                                      float64
Lagging_Current_Reactive.Power_kVarh           float64
Leading_Current_Reactive_Power_kVarh           float64
CO2(tCO2)                                      float64
Lagging_Current_Power_Factor                   float64
Leading_Current_Power_Factor                   float64
NSM                                              int64
WeekStatus                                      object
Day_of_week                                     object
Load_Type                                       object
dtype: object

In [9]:
# Crear nuevas columnas descomponiendo la fecha y hora en sus componentes
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['hour'] = df['date'].dt.hour
df['minute'] = df['date'].dt.minute
df['second'] = df['date'].dt.second
df['dayofweek'] = df['date'].dt.dayofweek  # Lunes=0, Domingo=6
df['dayofyear'] = df['date'].dt.dayofyear
df['weekofyear'] = df['date'].dt.isocalendar().week
df['quarter'] = df['date'].dt.quarter

In [10]:
df = pd.get_dummies(df, columns=["Day_of_week", "Load_Type"], drop_first=True)

In [11]:
df.columns.to_list()

['date',
 'Usage_kWh',
 'Lagging_Current_Reactive.Power_kVarh',
 'Leading_Current_Reactive_Power_kVarh',
 'CO2(tCO2)',
 'Lagging_Current_Power_Factor',
 'Leading_Current_Power_Factor',
 'NSM',
 'WeekStatus',
 'year',
 'month',
 'day',
 'hour',
 'minute',
 'second',
 'dayofweek',
 'dayofyear',
 'weekofyear',
 'quarter',
 'Day_of_week_Monday',
 'Day_of_week_Saturday',
 'Day_of_week_Sunday',
 'Day_of_week_Thursday',
 'Day_of_week_Tuesday',
 'Day_of_week_Wednesday',
 'Load_Type_Maximum_Load',
 'Load_Type_Medium_Load']

In [12]:
df['IsWeekend'] = df['WeekStatus'] == 'Weekend'

In [13]:
df = df.drop(['date', 'WeekStatus'], axis=1)

In [14]:
df.dtypes

Usage_kWh                               float64
Lagging_Current_Reactive.Power_kVarh    float64
Leading_Current_Reactive_Power_kVarh    float64
CO2(tCO2)                               float64
Lagging_Current_Power_Factor            float64
Leading_Current_Power_Factor            float64
NSM                                       int64
year                                      int32
month                                     int32
day                                       int32
hour                                      int32
minute                                    int32
second                                    int32
dayofweek                                 int32
dayofyear                                 int32
weekofyear                               UInt32
quarter                                   int32
Day_of_week_Monday                         bool
Day_of_week_Saturday                       bool
Day_of_week_Sunday                         bool
Day_of_week_Thursday                    

In [15]:
X = df
X = df.drop('Usage_kWh', axis=1)
y = df['Usage_kWh']

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear el modelo de regresión lineal
model = LinearRegression()

# Entrenar el modelo
model.fit(X_train, y_train)

# Realizar predicciones
y_pred = model.predict(X_test)

# Evaluar el modelo
# Calcular MSE
mse = mean_squared_error(y_test, y_pred)

# Calcular RMSE
rmse = np.sqrt(mse)

# Calcular MAE
mae = mean_absolute_error(y_test, y_pred)

# Calcular R^2
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'Mean Absolute Error (MAE): {mae}')
print(f'R^2 Score: {r2}')

Mean Squared Error (MSE): 17.717200552037106
Root Mean Squared Error (RMSE): 4.2091805083694265
Mean Absolute Error (MAE): 2.5952191751719313
R^2 Score: 0.9844137834167075


Agregamos modelos a utilizar, Arbol de Decisión, RandomForest, XGBoost Regressor, CUBIST

In [17]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

modelos = [('LinearRegression',LinearRegression()),
    ('DecisionTreeRegressor', DecisionTreeRegressor(max_depth=2)),
    ('RandomForestRegressor', RandomForestRegressor(min_samples_split=2, min_weight_fraction_leaf=0.0,n_estimators=40, n_jobs=-1, oob_score=True,random_state=None, verbose=0, warm_start=False)),
    ('XGBRegressor', xgb.XGBRegressor(learning_rate=0.01,n_estimators=500,max_depth=5,eval_metric='rmsle')),
    ('Cubist', Cubist(n_rules=500,neighbors=None,unbiased=True,auto=False,extrapolation=0.1,n_committees=5))
]


#procedimiento para comparar los modelos
for nombre, model in modelos:
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  mse = mean_squared_error(y_test, y_pred)
  rmse = np.sqrt(mse)
  mae = mean_absolute_error(y_test, y_pred)
  r2 = r2_score(y_test, y_pred)

  print(f'{nombre}," score: ",{model.score(X_test,y_pred):.03f}',end=" ")
  print(f'Mean Squared Error (MSE): {mse:.03f}',end=" ")
  print(f'Root Mean Squared Error (RMSE): {rmse:.03f}',end=" ")
  print(f'Mean Absolute Error (MAE): {mae:.03f}',end=" ")
  print(f'R^2 Score: {r2:.03f}',end=" ")
  print("")

LinearRegression," score: ",1.000 Mean Squared Error (MSE): 17.717 Root Mean Squared Error (RMSE): 4.209 Mean Absolute Error (MAE): 2.595 R^2 Score: 0.984 
DecisionTreeRegressor," score: ",1.000 Mean Squared Error (MSE): 65.380 Root Mean Squared Error (RMSE): 8.086 Mean Absolute Error (MAE): 4.597 R^2 Score: 0.942 
RandomForestRegressor," score: ",1.000 Mean Squared Error (MSE): 1.331 Root Mean Squared Error (RMSE): 1.154 Mean Absolute Error (MAE): 0.396 R^2 Score: 0.999 
XGBRegressor," score: ",1.000 Mean Squared Error (MSE): 5.291 Root Mean Squared Error (RMSE): 2.300 Mean Absolute Error (MAE): 1.352 R^2 Score: 0.995 
Cubist," score: ",1.000 Mean Squared Error (MSE): 0.107 Root Mean Squared Error (RMSE): 0.327 Mean Absolute Error (MAE): 0.064 R^2 Score: 1.000 


Buscamos mejores valores de hiperparámetros para los 2 modelos con mejores resultados, XGBRegressor y Cubist
1. XGBRegressor

In [18]:
from pprint import pprint
import numpy as np

n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_leaves = [0, 8, 10, 12, 16, 20]  # 0 equivale a 'no limit'
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)] + [None]


random_grid = {'n_estimators': n_estimators,
               'max_leaves': max_leaves,
               'max_depth': max_depth}

print('Los valores a probar en la búsqueda aleatoria son:')
pprint(random_grid)

print()
print('El número total de combinaciones de parámetros de entrenamiento es',
      len(random_grid['n_estimators']) *
      len(random_grid['max_leaves']) *
      len(random_grid['max_depth'])
      )

Los valores a probar en la búsqueda aleatoria son:
{'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None],
 'max_leaves': [0, 8, 10, 12, 16, 20],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

El número total de combinaciones de parámetros de entrenamiento es 720


Usaremos la recomendación de busqueda con el 10%

In [19]:
from sklearn.model_selection import RandomizedSearchCV

xgbb = xgb.XGBRegressor()
xgb_random = RandomizedSearchCV(estimator = xgbb,
                               param_distributions = random_grid,
                               n_iter = 20,
                               cv = 3,          # Validación cruzada 3-fold
                               verbose=2,
                               random_state=0,
                               n_jobs = -1      # Paralelizar en todos los cores disponibles
                               )
xgb_random.fit(X_train, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


RandomizedSearchCV(cv=3,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=...
                                          min_child_weight=None, missing=nan,
                                          monotone_constraints=None,
                                          multi_strategy=None,
                                          n_estimators=None, n_jobs=None,
                                          num_parallel_tree=None,
                                          random_state=None, ...),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_leaves': [0, 8, 10, 12, 16, 20],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=0, verbose=2)

In [20]:
xgb_random_best = xgb_random.best_estimator_

print('Los hiperparámetros del mejor modelo son:')
pprint(xgb_random.best_params_)
print()

print('Exactitud luego de búsqueda aleatoria en entrenamiento:', xgb_random_best.score(X_train, y_train))
print('Exactitud luego de búsqueda aleatoria en validación:', xgb_random_best.score(X_test, y_test))

Los hiperparámetros del mejor modelo son:
{'max_depth': 10, 'max_leaves': 8, 'n_estimators': 1000}

Exactitud luego de búsqueda aleatoria en entrenamiento: 0.9997227087327877
Exactitud luego de búsqueda aleatoria en validación: 0.9993498721698906


2. Búsqueda de hiperparámetros para el método Cubist

In [21]:
from pprint import pprint
import numpy as np

###Hiperparámetros a considerar
# n_rules (int, default=500)
# n_committees (int, default=0):
# neighbors (int, default=None)
# extrapolation (float, default=0.05):

n_rules = [int(x) for x in np.linspace(start = 200, stop = 1000, num = 10)]
n_committees = [1, 2, 5, 10, 15]  # 5 es el recomendado en la documentación
neighbors = [int(x) for x in np.linspace(1, 9, num = 1)]
extrapolation = [0.01, 0.03, 0.05, 0.07, 0.09] #0,05 = 5% es el recomendado


random_grid = {'n_rules': n_rules,
               'n_committees': n_committees,
               'neighbors': neighbors,
               'extrapolation':extrapolation}

print('Los valores a probar en la búsqueda aleatoria son:')
pprint(random_grid)

print()
print('El número total de combinaciones de parámetros de entrenamiento es',
      len(random_grid['n_rules']) *
      len(random_grid['n_committees']) *
      len(random_grid['neighbors']) *
      len(random_grid['extrapolation'])
      )

Los valores a probar en la búsqueda aleatoria son:
{'extrapolation': [0.01, 0.03, 0.05, 0.07, 0.09],
 'n_committees': [1, 2, 5, 10, 15],
 'n_rules': [200, 288, 377, 466, 555, 644, 733, 822, 911, 1000],
 'neighbors': [1]}

El número total de combinaciones de parámetros de entrenamiento es 250


In [22]:
#usamos la recomendación de probar con el 10%

cbst = Cubist()
cbst_random = RandomizedSearchCV(estimator = cbst,
                               param_distributions = random_grid,
                               n_iter = 5,
                               cv = 3,          # Validación cruzada 3-fold
                               verbose=2,
                               random_state=0,
                               n_jobs = -1      # Paralelizar en todos los cores disponibles
                               )
cbst_random.fit(X_train, y_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


RandomizedSearchCV(cv=3, estimator=Cubist(), n_iter=5, n_jobs=-1,
                   param_distributions={'extrapolation': [0.01, 0.03, 0.05,
                                                          0.07, 0.09],
                                        'n_committees': [1, 2, 5, 10, 15],
                                        'n_rules': [200, 288, 377, 466, 555,
                                                    644, 733, 822, 911, 1000],
                                        'neighbors': [1]},
                   random_state=0, verbose=2)

In [23]:
cbst_random_best = cbst_random.best_estimator_

print('Los hiperparámetros del mejor modelo son:')
pprint(cbst_random.best_params_)
print()

print('Exactitud luego de búsqueda aleatoria en entrenamiento:', cbst_random_best.score(X_train, y_train))
print('Exactitud luego de búsqueda aleatoria en validación:', cbst_random_best.score(X_test, y_test))

Los hiperparámetros del mejor modelo son:
{'extrapolation': 0.03, 'n_committees': 15, 'n_rules': 377, 'neighbors': 1}

Exactitud luego de búsqueda aleatoria en entrenamiento: 0.999999999999999
Exactitud luego de búsqueda aleatoria en validación: 0.9999029658252098


**Uso de la librería OPTUNA para encontrar los mejores hiperparámetros**

Estimamos los mejores hiperparámetros usando la librería OPTUNA sugerida en el plantemiento del proyecto

In [24]:
import optuna
import optuna.visualization as vis

# Para trabajar con optuna, debemos definir la función objetivo que luego le diremos a optuna que debe minimizar (según nuestro indicador escogido RMSE)
def objective(trial):
  # Dejamos que optuna sugiera los hiperparámetros iniciales
  n_rules = trial.suggest_int("n_rules", 200, 1000)
  n_committees = trial.suggest_int("n_committees", 1, 15)
  neighbors = trial.suggest_int("neighbors", 1, 9)
  extrapolation = trial.suggest_float("extrapolation",0.01, 0.09)

  #Definimos el modelo Cubist a utilizar para la optimización
  cbst = Cubist(n_rules= n_rules, n_committees=n_committees, neighbors=neighbors, extrapolation=extrapolation)
  cbst.fit(X_train, y_train)

  # Make predictions and calculate RMSE
  y_pred = model.predict(X_test)
  rmse = np.sqrt(mean_squared_error(y_test, y_pred))
  mae = mean_absolute_error(y_test, y_pred)
  r2 = r2_score(y_test, y_pred)

  # Return MAE
  return mae

In [25]:
# Creamos el "objeto de estudio", como llama Optuna
study = optuna.create_study(direction="minimize")

# Ejecutamos el proceso en si de optimización, le pasamos la función objetivo definida en el paso previo
study.optimize(objective, n_trials=10, show_progress_bar=True)

[I 2024-06-17 17:21:00,928] A new study created in memory with name: no-name-753df7b1-5953-4688-b22c-008f6c067592


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2024-06-17 17:21:36,006] Trial 0 finished with value: 0.06420718489852668 and parameters: {'n_rules': 887, 'n_committees': 14, 'neighbors': 3, 'extrapolation': 0.01920184738811078}. Best is trial 0 with value: 0.06420718489852668.
[I 2024-06-17 17:22:09,938] Trial 1 finished with value: 0.06420718489852668 and parameters: {'n_rules': 378, 'n_committees': 14, 'neighbors': 2, 'extrapolation': 0.08678476587276159}. Best is trial 0 with value: 0.06420718489852668.
[I 2024-06-17 17:22:17,610] Trial 2 finished with value: 0.06420718489852668 and parameters: {'n_rules': 867, 'n_committees': 2, 'neighbors': 2, 'extrapolation': 0.03615188954124826}. Best is trial 0 with value: 0.06420718489852668.
[I 2024-06-17 17:23:02,190] Trial 3 finished with value: 0.06420718489852668 and parameters: {'n_rules': 941, 'n_committees': 15, 'neighbors': 4, 'extrapolation': 0.07855542457434765}. Best is trial 0 with value: 0.06420718489852668.
[I 2024-06-17 17:23:07,599] Trial 4 finished with value: 0.064207

In [26]:
# Imprimimos los resultados del estudio con optuna
print("Best trial:", study.best_trial)
print("Best hyperparameters:", study.best_params)

Best trial: FrozenTrial(number=0, state=1, values=[0.06420718489852668], datetime_start=datetime.datetime(2024, 6, 17, 17, 21, 0, 934521), datetime_complete=datetime.datetime(2024, 6, 17, 17, 21, 36, 6493), params={'n_rules': 887, 'n_committees': 14, 'neighbors': 3, 'extrapolation': 0.01920184738811078}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_rules': IntDistribution(high=1000, log=False, low=200, step=1), 'n_committees': IntDistribution(high=15, log=False, low=1, step=1), 'neighbors': IntDistribution(high=9, log=False, low=1, step=1), 'extrapolation': FloatDistribution(high=0.09, log=False, low=0.01, step=None)}, trial_id=0, value=None)
Best hyperparameters: {'n_rules': 887, 'n_committees': 14, 'neighbors': 3, 'extrapolation': 0.01920184738811078}


In [27]:
#Utilizamos algunos gráficos utiles de la librería optuna
# Plotear el historial de la optimización realizada
vis.plot_optimization_history(study)

# Plotear tipo 'slice'
#vis.plot_slice(study, params=["n_rules", "n_committees"])

# Ploteo tipo 'contorno'
#vis.plot_contour(study, params=["neighbors", "extrapolation"])

# Plotear parallel_coordinate
vis.plot_parallel_coordinate(study)

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.

In [ ]:
optuna.visualization.plot_optimization_history(study)